# Load models to run predictions

### Run helper functions to generate the data again

If you want, you can always save out the dictionaries and the Loaders, but when it takes only 30 seconds to generate them, and this is just for fun, it would probably just be a waste of storage

In [ ]:
%run ./helper_functions.ipynb

### Specifics to the model in question
We'll need all the info we originally had to prepare the data and save out the file, like `data_folder`, `save_folder`, `model_name`, `embedding_dim` and the data and batch sizes.  In addition to what we had before, we need the `epoch` (int) that we want to load

In [ ]:
data_folder = "./mozart_sonatas/"
save_folder = 'V4'
model_name = "lstm_model"
epoch = 30
data_size = 150
batch_size = 256*4
embedding_dim = 32

filepath = f"./{save_folder}/{model_name}_epoch{epoch}.mid"

### Generate the tensors / dictionaries again

In [ ]:
raw_notes, raw_offsets, raw_length = music_to_lists(data_folder)
notes, offsets, length = clean_lists(raw_notes, raw_offsets, raw_length)

note_dict = make_unique_map(notes)
len_dict = make_unique_map(length)
offset_dict = make_unique_map(offsets)

num_len = [len_dict[round(i,4)] for i in length]
num_notes = [note_dict[i] for i in notes]
num_offset = [offset_dict[round(i,4)] for i in offsets]

#Create model for 
TestLoader, TrainLoader, X_train, X_test, y_train, y_test = loadData(data_size, batch_size, [num_notes, num_offset, num_len])
for i in range(len(y_train)):
    y_train[i][1] += len(note_dict)
    y_train[i][2] += len(offset_dict) + len(note_dict)
for i in range(len(y_test)):
    y_test[i][1] += len(note_dict)
    y_test[i][2] += len(offset_dict) + len(note_dict)
    
sizes = (len(set(num_notes)),len(set(num_offset)),len(set(num_len)))

### Fun part: Load the model

The below function not only returns the loaded model, but also the objective function and the optimizer.  That means, if you want to keep training, you can!  Otherwise, it is set to `.eval()` before being returned as a precaution since it will likely just be used for modeling

In [ ]:
def load_RNN(filepath, embedding_dim, sizes):
    # instatiate RNN object
    model = RNN(sizes,embedding_dim = embedding_dim)
    
    # load dictionary of params
    checkpoint = torch.load(filepath)
    
    # load attributes and instantiate objective + optimizer
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    last_epoch = checkpoint['epoch'] # just to check that its the right one
    last_loss = checkpoint['loss']
    objective = nn.BCEWithLogitsLoss()
    
    model.eval()

    return model, objective, optimizer, last_loss, last_epoch

model, objective, optimizer, _, _ = load_RNN(filepath, embedding_dim, sizes)

In [ ]:
tester = list(TestLoader) #points to test
rand_index = np.random.randint(0,len(tester)) #choose an index
reverse_mapping = {"note" : {note_dict[k]:k for k in note_dict},
                   "off" : {offset_dict[k]:k for k in offset_dict},
                   "len" : {len_dict[k]:k for k in len_dict}}

In [ ]:
def predict(more_notes, sizes, model, criterion, optimizer, test_loader, reverse_mapping, rand_choice=None):
    preds = []
    if rand_choice is None: #
        rand_choice = np.random.randint(0,len(tester))
    rand_index = np.random.randint(0,127)
    temp_data = tester[rand_choice][0][rand_index].reshape(1,-1)
    og_data = temp_data
    H = model.initHidden(1) #get the hidden states
    
    while len(preds) < more_notes*len(sizes):
        
        output, H = model(temp_data, H) #find the output
        p = torch.argmax(output[0][:sizes[0]]) #find the location of the largest
        q = torch.argmax(output[0][sizes[0]:sizes[0] + sizes[1]])
        r = torch.argmax(output[0][sizes[0] + sizes[1]:])
        p, q, r = torch.flatten(p),torch.flatten(q+sizes[0]),torch.flatten(r+sizes[0] +sizes[1])
        preds += list(p.numpy()) + list(q.numpy()) + list(r.numpy()) #get the predictions
        temp_data = torch.cat((temp_data.squeeze(),p,q,r))
        temp_data = temp_data[len(sizes):].unsqueeze(0).reshape(1,-1)

    pred_note = [reverse_mapping["note"][p] for p in og_data[:rand_index].numpy()[0][::3]] + [reverse_mapping["note"][p] for p in preds[::3]]
    pred_off = [reverse_mapping["off"][p-sizes[0]] for p in og_data[:rand_index].numpy()[0][1::3]] + [reverse_mapping["off"][p-sizes[0]] for p in preds[1::3]]
    pred_len = [reverse_mapping["len"][p-sizes[0] - sizes[1]] for p in og_data[:rand_index].numpy()[0][2::3]] + [reverse_mapping["len"][p-sizes[0]-sizes[1]] for p in preds[2::3]]
    
    
    return pred_note, pred_off, pred_len
more_notes = 600
pred_note, pred_off, pred_len = predict(more_notes, sizes, model, objective, optimizer, TestLoader, reverse_mapping)
len(pred_note), len(pred_off), len(pred_len)

In [ ]:
new_comp = []
previous_offset = 0
for i in range(len(pred_note)):

    if '.' in pred_note[i]:
        chord_pitches = pred_note[i].split('.')
        notes = []
        for pitch in chord_pitches:
            new_note = note.Note(pitch)
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = pred_off[i] + previous_offset
        new_chord.duration.quarterLength = pred_len[i]
        new_comp.append(new_chord)
    
    else:
        new_note = note.Note(pred_note[i])
        new_note.storedInstrument = instrument.Piano()
        new_note.offset = pred_off[i] + previous_offset
        new_note.duration.quarterLength = pred_len[i]
        new_comp.append(new_note)
    
    previous_offset += pred_off[i]

midi_stream = stream.Stream(new_comp)
midi_stream.write('midi', fp='song.mid')

In [ ]:
new_comp